In [6]:
import torch
import torch.nn as nn
from torchtext import data

In [5]:
train_data, test_data

Please install SpaCy. See the docs at https://spacy.io for more information.


ModuleNotFoundError: No module named 'spacy'